In [ ]:
import torch
from torch import optim,nn
import torch.nn.functional as F
from torch.autograd import Variable
import gc
import numpy as np
import time
import os
import random
import pickle
import argparse
from functions import get_dates,calculate
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import average_precision_score as AP

In [ ]:
from models.gru_bidirectional import GRU
from models.retain_bidirectional import RETAIN
from models.retain_ex import RETAIN_EX

In [ ]:
ver_list = ['retain','ex','gru']
task_list = ['I50','H26']
time_list = [0,1]
hid_list = [64,128,256]
lr_list = [0.01, 0.001, 0.0001]
epoch_list = np.arange(30)+1

In [ ]:
ver='gru'
lr = 0.01
hid = 128
task = 'I50'
if ver=='gru':
    from models.gru_bidirectional import GRU
with open('data/%s/val.pckl'%task,'rb') as f:
    val_data = pickle.load(f)
model = GRU(hid,hid,1,cuda_flag=True)
model.load_state_dict(torch.load('experiments/H26/saved_weights/lr_0.001/retain_time_interpret/epoch/10_epochs.pth'))
# model.load_state_dict(torch.load('experiments2/I50/gru_128_0.0100/saved_weights/2_cuda.pckl'))
model.cuda()

In [ ]:
print('task\tver\ttime\thid\tlr\tepoch\tauc\tap')
for task in task_list:
    with open('data/%s/val.pckl'%task,'rb') as f:
        val_data = pickle.load(f)
#     for hid in [128]:
    for hid in hid_list:
            for time_fn in time_list:
                for ver in ver_list:
                        if (time_fn==1)&(ver!='ex'):
                            continue
                        else:
                            if ver=='gru':
                                model = GRU(hid,hid,1,cuda_flag=True)
                            elif ver=='retain':
                                model = RETAIN(hid,hid,1,cuda_flag=True)
                            elif (ver=='ex'):
                                model = RETAIN_EX(hid,hid,1,cuda_flag=True,bidirectional=True,time_ver=time_fn)
                            model.cuda()    
                            for lr in lr_list:
            #                 for lr in ['0.001']:
                                for epoch in epoch_list[:10]:
            #                     for epoch in [2]:
                                    if ver=='ex':
                                        weight_name = 'experiments2/%s/%s-%d_%d_%s/saved_weights/%d_cpu.pckl'%(task,ver,time_fn,hid,str(lr),epoch)
                                    else:
                                        weight_name = 'experiments2/%s/%s_%d_%s/saved_weights/%d_cpu.pckl'%(task,ver,hid,lr,epoch)
                                    try:
                                        model.load_state_dict(torch.load(weight_name))
                                        correct_list = []
                                        score_list = []
                                        for (X,y) in val_data:
                                            outputs,targets = calculate(X,y,model,ver=ver,cuda_flag=True,time_fn=time_fn)
                                            correct_list.extend(y)
                                            score_list.extend(outputs.squeeze().data.tolist())
                                        auc = AUC(y_true=correct_list,y_score=score_list)
                                        ap = AP(y_true=correct_list,y_score=score_list)
                                        print('%s\t%s\t%d\t%d\t%s\t%d\t%1.3f\t%1.3f'%(task,ver,time_fn,hid,str(lr),epoch,auc,ap))
                                    except FileNotFoundError:
            #                             print('file not found')
                                        continue

In [ ]:
weight_name